In [2]:
import cv2
import numpy as np
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

def detect_skin(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 30, 60], dtype=np.uint8)
    upper_skin = np.array([20, 150, 255], dtype=np.uint8)
    skin_mask = cv2.inRange(hsv_frame, lower_skin, upper_skin)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_CLOSE, kernel)
    return skin_mask

def process_frame(frame, volume_control):

    skin_mask = detect_skin(frame)
    skin_edges = cv2.Canny(skin_mask, 50, 150)
    contours, _ = cv2.findContours(skin_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        cv2.putText(frame, "No contours detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        return frame, None
    max_contour = max(contours, key=cv2.contourArea)
    contour_area = cv2.contourArea(max_contour)
    if contour_area < 100:
        cv2.putText(frame, "Contour too small", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        return frame, None
    hull = cv2.convexHull(max_contour, returnPoints=False)
    hull_points = cv2.convexHull(max_contour, returnPoints=True)
    M = cv2.moments(hull_points)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        centroid = (cx, cy)
        cv2.circle(frame, centroid, 5, (255, 0, 0), -1)
        cv2.putText(frame, f"Centroid: ({cx}, {cy})", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    else:
        centroid = None
    fingertips = []
    thumb_tip = None  
    index_tip = None  
    if len(hull) >= 3:
        defects = cv2.convexityDefects(max_contour, hull)
        if defects is not None:
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(max_contour[s][0])
                end = tuple(max_contour[e][0])
                far = tuple(max_contour[f][0])
                a = np.linalg.norm(np.array(start) - np.array(far))
                b = np.linalg.norm(np.array(end) - np.array(far))
                c = np.linalg.norm(np.array(start) - np.array(end))
                angle = np.arccos((a**2 + b**2 - c**2) / (2 * a * b))

                if angle < np.pi / 1.6 and d > 5000:
                    fingertips.append(start)
                    cv2.circle(frame, start, 8, (0, 255, 0), -1) 
                    if f == 4: 
                        thumb_tip = far
                    elif f == 8:  
                        index_tip = far
    if thumb_tip is not None and index_tip is not None:
        cv2.line(frame, thumb_tip, index_tip, (255, 0, 0), 2) 
        length = np.linalg.norm(np.array(thumb_tip) - np.array(index_tip))  
        cv2.putText(frame, f"Distance: {length:.2f}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
        volume = np.interp(length, [100, 300], [0, 1])  
        volume_control.SetMasterVolumeLevelScalar(volume, None)
        volume_bar_width = int(500 * volume)  
        cv2.rectangle(frame, (10, 200), (10 + volume_bar_width, 220), (0, 255, 0), -1) 
    cv2.drawContours(frame, [hull_points], -1, (0, 255, 255), 2)
    cv2.putText(frame, f"Hull Points: {len(hull_points)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
    cv2.putText(frame, f"Contour Area: {contour_area:.2f}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
    return frame, fingertips

def main():
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(
        IAudioEndpointVolume._iid_, 1, None)
    volume_control = interface.QueryInterface(IAudioEndpointVolume)
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        try:
            frame = cv2.flip(frame, 1)
            processed_frame, fingertips = process_frame(frame, volume_control)
            if fingertips:
                cv2.putText(processed_frame, f"Fingertips: {len(fingertips)}", (10, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
            cv2.imshow("Fingertip Detection with Skin Detection", processed_frame)
            if cv2.waitKey(1) & 0xFF == 27:  
                break
        except Exception as e:
            print(f"Error in processing frame: {e}")

    cap.release()
    cv2.destroyAllWindows()
    
if __name__ == "__main__":
    main() 

Error in processing frame: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'cv::convexityDefects'

Error in processing frame: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'cv::convexityDefects'

Error in processing frame: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'cv::convexityDefects'

Error in processing frame: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc

In [ ]:
import os
import cv2
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix

def load_ground_truth(ground_truth_file):
    import json
    with open(ground_truth_file, "r") as file:
        ground_truth = json.load(file)
    return ground_truth

def detect_skin(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 30, 60], dtype=np.uint8)
    upper_skin = np.array([20, 150, 255], dtype=np.uint8)
    skin_mask = cv2.inRange(hsv_frame, lower_skin, upper_skin)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_CLOSE, kernel)
    return skin_mask

def process_frame(frame):
    skin_mask = detect_skin(frame)
    skin_edges = cv2.Canny(skin_mask, 50, 150)
    contours, _ = cv2.findContours(skin_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return 0  
    max_contour = max(contours, key=cv2.contourArea)
    contour_area = cv2.contourArea(max_contour)
    if contour_area < 100:
        return 0  
    return 1  

def evaluate_dataset(dataset_folder, ground_truth_file):
    ground_truth = load_ground_truth(ground_truth_file)
    y_true = []
    y_pred = []

    for image_file in os.listdir(dataset_folder):
        image_path = os.path.join(dataset_folder, image_file)
        image = cv2.imread(image_path)
        if image is None:
            continue

        true_label = ground_truth.get(image_file, 0)
        y_true.append(true_label)

        pred_label = process_frame(image)
        y_pred.append(pred_label)

    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    report = classification_report(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    tp = conf_matrix[1, 1]  
    fn = conf_matrix[1, 0] 
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0

    print("=========================================================")
    print("Classification Report:")
    print(report)
    print("Confusion Matrix:")
    print(conf_matrix)
    print("=========================================================")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Detection Rate: {detection_rate:.2f}")
    print("=========================================================")

if __name__ == "__main__":
    dataset_folder = "Dataset/Mix/"  
    ground_truth_file = "ground_truth.json"  
    evaluate_dataset(dataset_folder, ground_truth_file)